# Teste 1 - Python

        Tentando importar e utilizar os dados do Ministério da Saúde em https://covid.saude.org.br para avaliar a situação divulgada oficialmente pelo MS

## Gráficos dos Dados

        Mostrar infectados e mortes confirmados diariamente
        Mostrar infectados e mortes confirmados acumulados
        Mostrar dados separados por região e por estado

## Estimativas (Regressão)
        
        Estimar próximos números (até 7 dias?) utilizando 2, 3 ou 4 (todos?) dias anteriores e regressão não linear
        
## Modelo de Contágio

        Utilizar dados adquiridos para rodar um modelo epidemiológico que utiliza valores de reprodução base e evolução da COVID-19, causada pelo vírus SARS-COV-2

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
df = pd.read_csv("COVID19_20200420.csv", sep = ";")
df.groupby('data').sum() ['casosNovos']

data
2020-01-30       0
2020-01-31       0
2020-02-01       0
2020-02-02       0
2020-02-03       0
2020-02-04       0
2020-02-05       0
2020-02-06       0
2020-02-07       0
2020-02-08       0
2020-02-09       0
2020-02-10       0
2020-02-11       0
2020-02-12       0
2020-02-13       0
2020-02-14       0
2020-02-15       0
2020-02-16       0
2020-02-17       0
2020-02-18       0
2020-02-19       0
2020-02-20       0
2020-02-21       0
2020-02-22       0
2020-02-23       0
2020-02-24       0
2020-02-25       0
2020-02-26       1
2020-02-27       0
2020-02-28       0
              ... 
2020-03-22     418
2020-03-23     345
2020-03-24     310
2020-03-25     232
2020-03-26     482
2020-03-27     502
2020-03-28     486
2020-03-29     353
2020-03-30     323
2020-03-31    1138
2020-04-01    1117
2020-04-02    1076
2020-04-03    1146
2020-04-04    1222
2020-04-05     852
2020-04-06     926
2020-04-07    1661
2020-04-08    2210
2020-04-09    1930
2020-04-10    1781
2020-04-11    1089
2020-04

In [30]:
# df.groupby("data").sum() ["casosNovos"].sort_values()
# df.plot.scatter (x = 'data', y = 'casosNovos')
